In [3]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:

import json
import numpy as np
from pprint import pprint


labels = [u'beauty', u'bills', u'grocery', u'prescription', u'recharge', u'Refunds', u'Referral', u'support', u'offers']

from extract_data import extract_api_ai_data



label2text, max_sent_length = extract_api_ai_data(labels)

beauty [u'book beauty services', u'i want to book beauty services', u'book facial for me', u'how to book beauty services', u'book waxing service for me'] 

bills [u'pay my bill', u'pay my dth bill', u'pay my electricity bill', u'pay my water bills', u'bill payment', u'how do i pay bill', u'pay bills'] 

grocery [u'order', u'buy', u'order', u'how to order', u'show me the list to buy grocery'] 

prescription [u'order prescription medicine', u'order medicine', u'i want medicines for my prescription', u'how to order medicine', u'you deliver medicine'] 

recharge [u'recharge my phone', u'recharge phone', u'pay postpaid bill', u'rebharge', u'rehcarge', u'can you help reharge my phone', u'how recharge', u'how do i recharge', u'how recharge', u'how to recharge mobile phone', u'i want to recharge mobile phone', u'how to recharge here', u'how to recharge in this app', u'how can i recharge from this app', u'your airtel', u'your vodafone'] 

Refunds [u'refunds', u'when can i get refund', u'how wil

In [5]:
#http://scikit-learn.org/stable/tutorial/text_analytics/working_with_text_data.html

allsents = []
alllabels = []
label2onesent = {}

for label, sents in label2text.viewitems():
    #allsents.extend(sents)
    label2onesent[label] = ' '.join(sents)
    for sent in sents:
        allsents.append(sent)
        alllabels.append(label)


In [30]:
import nltk
from nltk.stem.porter import PorterStemmer
import string

def tokenize(text):
    tokens = nltk.word_tokenize(text)
    stems = []
    for item in tokens:
        if item in string.punctuation:
            #print item
            continue
        stems.append(PorterStemmer().stem(item))
    return stems

from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(stop_words='english',tokenizer=tokenize, ngram_range=(1, 2))
#count_vect = CountVectorizer(stop_words='english', ngram_range=(1, 2))

X_train_counts = count_vect.fit_transform(allsents)
print X_train_counts.shape
print count_vect.vocabulary_ #['give back']

#tfidf = TfidfVectorizer(stop_words='english')
#tfs = tfidf.fit_transform(X_train_counts)
from sklearn.feature_extraction.text import TfidfTransformer
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
print X_train_tfidf.shape

(80, 106)
{u'refund': 76, u'medicin': 42, u'rehcarg': 80, u'rebharg': 67, u'help': 35, u'money': 46, u'app': 4, u'list buy': 41, u'ani offer': 3, u'cancel': 17, u'thi app': 87, u'mobil': 44, u'wax servic': 105, u'fail transact': 33, u'order prescript': 55, u'initi refund': 39, u'recharg thi': 71, u'transact': 88, u'prescript': 64, u'avail': 5, u'avail referr': 7, u'cashback': 19, u'vodafon': 95, u'wax': 104, u'want cancel': 99, u'type': 91, u'type offer': 92, u'book': 11, u'recharg mobil': 69, u'help order': 36, u'offer provid': 52, u'prescript medicin': 65, u'offer': 51, u'avail offer': 6, u'electr': 30, u'reharg': 78, u'initi': 38, u'referr bonu': 75, u'did succeed': 27, u'beauti servic': 9, u'order medicin': 54, u'facial': 31, u'want book': 98, u'deliv medicin': 25, u'want medicin': 100, u'buy groceri': 16, u'earn': 29, u'dth': 28, u'reharg phone': 79, u'did': 26, u'servic': 81, u'need help': 48, u'bonu': 10, u'provid': 66, u'need money': 49, u'airtel': 0, u'medicin prescript': 43, 

In [37]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import SGDClassifier
text_clf = Pipeline([('vect', CountVectorizer(tokenizer=tokenize, ngram_range=(1, 2))), #stop_words='english',
                    ('tfidf', TfidfTransformer()),
                      ('clf', SGDClassifier(loss='hinge', penalty='l2',
                                           alpha=1e-3, n_iter=5, random_state=42)),
])
_ = text_clf.fit(allsents, alllabels)
predicted = text_clf.predict(allsents)
np.mean(predicted == alllabels)   

1.0

In [33]:
from sklearn import metrics
print(metrics.classification_report(alllabels, predicted,target_names=labels))
metrics.confusion_matrix(alllabels, predicted)

              precision    recall  f1-score   support

      beauty       1.00      1.00      1.00         9
       bills       1.00      1.00      1.00        18
     grocery       1.00      1.00      1.00         5
prescription       1.00      1.00      1.00         7
    recharge       1.00      1.00      1.00         5
     Refunds       1.00      1.00      1.00        11
    Referral       1.00      1.00      1.00         5
     support       1.00      1.00      1.00        16
      offers       1.00      1.00      1.00         4

 avg / total       1.00      1.00      1.00        80



array([[ 9,  0,  0,  0,  0,  0,  0,  0,  0],
       [ 0, 18,  0,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  5,  0,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  7,  0,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  5,  0,  0,  0,  0],
       [ 0,  0,  0,  0,  0, 11,  0,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  5,  0,  0],
       [ 0,  0,  0,  0,  0,  0,  0, 16,  0],
       [ 0,  0,  0,  0,  0,  0,  0,  0,  4]])

In [34]:
s = [
    'please recharge',
    ' book wax me'
]

sents = label2text['Refunds']
p = text_clf.predict(sents)
print p

[u'Refunds' u'Refunds' u'Refunds' u'Refunds' u'Refunds' u'Refunds'
 u'Refunds' u'Refunds' u'Refunds' u'Refunds' u'Refunds' u'Refunds'
 u'Refunds' u'Refunds' u'Refunds' u'Refunds' u'Refunds' u'Refunds']
